Das Jupyter Notebook muss mit Python >= 3.10 ausgeführt werden (getestet in 3.12)

In [1]:
!pip install gymnasium
!pip install numpy
!pip install pygame # todo braucht man pygame wirklich?


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [2]:
!pip install torch


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [7]:
import torch

print(torch.__version__)

2.5.1+cpu


In [2]:
!git clone https://github.com/lychanl/Gymnasium_Snake_Game

Cloning into 'Gymnasium_Snake_Game'...


In [3]:
import sys

sys.path.append('./Gymnasium_Snake_Game') # python erkennt das neu angelegte Verzeichnis manchmal nicht, deswegen hier explixit laden

Der Code ist von https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

In [96]:
import random
from collections import namedtuple, deque

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [97]:
import torch.nn as nn
import torch.nn.functional as F

class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        # x = x.view(x.size(0), -1)  # Input flatten (batch size, höhe,breite,rgb)
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [115]:
def map_state_to_array(state):
    snake_head = None
    blocks = []
    food = None
    direction = [0, 1, 2, 3]

    # Groesse des Spielfelds definieren
    height, width = state.shape[0], state.shape[1]  # (15, 15, 3)
    # height, width = state.shape[1], state.shape[2]  # (0, 15, 15, 3)

    # Position aller Elemente des Spielfelds speichern
    for y in range(height):
        for x in range(width):
            if state[y, x, 0] == 1:  # Snakes kopf
                snake_head = (x, y)
            elif state[y, x, 2] == 1:  # Food
                food = (x, y)
            elif state[y, x, 1] == 1:  # Bloecke
                blocks.append((x, y))
            # if state[0, y, x, 0] == 1:  # Snakes kopf
            #     snake_head = (x, y)
            # elif state[0, y, x, 1] == 1:  # Bloecke
            #     blocks.append((x, y))
            # elif state[0, y, x, 2] == 1:  # Food
            #     food = (x, y)
    
    result = np.zeros(20)

    # todo bei den bloecken evtl. noch das ende des Spielfelds einbeziehen?
    # 1-4: Schauen, ob sich geradeaus vor dem Kopf der Schlange ein Block befindet (oder rechts, links, dahinter)
    result[0:4] = [0] * 4
    if snake_head:
        for i, (dx, dy) in enumerate([(0, -1), (1, 0), (0, 1), (-1, 0)]):
            next_x = snake_head[0] + dx
            next_y = snake_head[1] + dy
            if 0 <= next_x < width and 0 <= next_y < height:
                if (next_x, next_y) in blocks:
                    result[i] = 1

    # 5-8: Schauen, ob sich rechs neben dem Kopf der Schlange ein Block befindet
    result[4:8] = [0] * 4
    if snake_head:
        for i, (dx, dy) in enumerate([(0, -1), (1, 0), (0, 1), (-1, 0)]):
            next_x = snake_head[0] + dx
            next_y = snake_head[1] + dy
            right_x = next_x + dy
            right_y = next_y - dx
            if 0 <= right_x < width and 0 <= right_y < height:
                if (right_x, right_y) in blocks:
                    result[4 + i] = 1

    # 9-12: Schauen, ob sich links neben dem Kopf der Schlange ein Block befindet
    result[8:12] = [0] * 4
    if snake_head:
        for i, (dx, dy) in enumerate([(0, -1), (1, 0), (0, 1), (-1, 0)]):
            next_x = snake_head[0] + dx
            next_y = snake_head[1] + dy
            left_x = next_x - dy
            left_y = next_y + dx
            if 0 <= left_x < width and 0 <= left_y < height:
                if (left_x, left_y) in blocks:
                    result[8 + i] = 1

    result[12:16] = direction

    # 17-20: Position des Essens relativ zu dem Kopf der Schlange ermitteln
    if food:
        result[16] = 1 if food[0] < snake_head[0] else 0  # food links
        result[17] = 1 if food[0] > snake_head[0] else 0  # food rechts
        result[18] = 1 if food[1] < snake_head[1] else 0  # food oben
        result[19] = 1 if food[1] > snake_head[1] else 0  # food unten

    return result

In [110]:
import gymnasium_snake_game
import gymnasium
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt

from IPython import display

import torch
import torch.optim as optim

# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Falls das Jupyter Notebook 
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)


options = {
    'fps': 30,
    'max_step': 1500,
    'init_length': 4,
    'food_reward': 40.0,
    'dist_reward': 10.0, # je naeher die Schlange am Essen ist (euklidische Distanz von Schlange zum essen)
    'living_bonus': 0.1,
    'death_penalty': -50.0,
    'width': 15,
    'height': 15,
}

env = gymnasium.make('Snake-v1', render_mode='human', **options)


# env.reset() # braucht man nicht!?


# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
state = map_state_to_array(state)
n_observations = len(state)


# n_observations = 40 * 40 * 3 # höhe, breite, rgb farben
# n_observations = state.shape[0] * state.shape[1] * state.shape[2]

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            # return policy_net(state).max(1).indices.view(1, 1)
            return policy_net(state).argmax(dim=1).view(1, 1)
    else:
        # return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)
        return torch.tensor([[random.randint(0, 3)]], device=device, dtype=torch.long)

episode_scores = []


def plot_scores(show_result=False):
    plt.figure(1)
    scores_t = torch.tensor(episode_scores, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.plot(scores_t.numpy())
    # Take 100 episode averages and plot them too
    if len(scores_t) >= 100:
        means = scores_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [111]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)    
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    print(batch.next_state)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [117]:
from itertools import count

# if torch.cuda.is_available() or torch.backends.mps.is_available():
#     num_episodes = 600
# else:
#     num_episodes = 50
num_episodes = 5000

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = map_state_to_array(state)
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    score = 0  # Score der Snake zu Beginn
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        # observation returned ein Array (15,15,3) (höhe,breite, (1=Sneak Head, 2=Blocks, 3=Food))
        score += reward
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            new_observation = map_state_to_array(observation)
            next_state = torch.tensor(new_observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        # next_state = map_state_to_array(next_state)
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        # optimize_model() # todo

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_scores.append(score)
            plot_scores()
            break

print('Complete')
plot_scores(show_result=True)
plt.ioff()
plt.show()

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [1]:
!pip install torch torchvision 

   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/203.0 MB 2.0 MB/s eta 0:01:43
   ---------------------------------------- 0.4/203.0 MB 5.0 MB/s eta 0:00:41
   ---------------------------------------- 1.4/203.0 MB 9.9 MB/s eta 0:00:21
    --------------------------------------- 2.9/203.0 MB 15.5 MB/s eta 0:00:13
    --------------------------------------- 4.1/203.0 MB 18.8 MB/s eta 0:00:11
    --------------------------------------- 4.1/203.0 MB 18.8 MB/s eta 0:00:11
   - -------------------------------------- 6.1/203.0 MB 18.5 MB/s eta 0:00:11
   - -------------------------------------- 8.2/203.0 MB 21.9 MB/s eta 0:00:09
   - -------------------------------------- 9.0/203.0 MB 21.2 MB/s eta 0:00:10
   -- ------------------------------------- 12.0/203.0 MB 32.7 MB/s eta 0:00:06
   -- ------------------------------------- 13.6/203.0 MB 32.7 MB/s eta 0:00:06
   -- ------------------------------------- 15.1/203.0 MB 38.


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Kai\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip
